# ArcSOC Optimizer

The following code takes ArcSOC Optimizer outputs and organizes the information.

When multiple CSVs are provided (input_names), the first CSV is analyzed. The others are diff'ed with the first one to see if there are discrepancies among the recommendations.

In [ ]:
import pandas as pd
from datetime import datetime
import os
from os import path
from IPython.display import display

In [ ]:
print(f'Run on {datetime.now()} by {os.getlogin()}')

In [ ]:
# Assuming files are in the ArcSOC_Optimizer directory
# First file will be compared to all others for detecting differences
OPTIMIZER_DIR = 'ArcSOC_Optimizer'
input_names = [ "ags.ecomm911.ca_default_30_days_2022_05_10_10_17_36.csv",
               "ags.ecomm911.ca_default_1_days_2022_05_12_05_02_37.csv",
               "ags.ecomm911.ca_default_1_days_2022_05_11_05_02_32.csv", 
               "ags.ecomm911.ca_default_1_days_2022_05_10_10_21_04.csv" ]
input_csv = []
for n in input_names:
    p = path.join(OPTIMIZER_DIR, n)
    print(f'[{len(input_csv)}]: {p}')
    input_csv.append(pd.read_csv(p, skiprows=11))


In [ ]:
# Filter out stopped services
for i in range(0, len(input_csv)):
    input_csv[i] = input_csv[i][(input_csv[i]['comment'].str.contains('stopped')==False)]

In [ ]:
delta_with_usage = []
delta = []
for i in range(0, len(input_csv)):
    d = input_csv[i][['service','usageSec','minOld','minNew','maxOld','maxNew']]
    delta_with_usage.append(d.set_index('service'))
    # This delta is only used for the diff operation
    d = input_csv[i][['service','minOld','minNew','maxOld','maxNew']]
    delta.append(d)

# df references the first CSV's data in the min/max analyses
df = delta_with_usage[0]


## All Changes

In [ ]:
#change = df.loc[(df['minOld']!=df['minNew']) | (df['maxOld']!=df['maxNew'])]
#change

## Differences between the first CSV and others

In [ ]:
# Diff the first with other CSVs
# This is the only place where "delta" is used
for i in range(1, len(delta)):
    print(f'diff with {input_names[i]}')
    df1_str_tuples = delta[0].astype(str).apply(tuple, 1)
    df2_str_tuples = delta[i].astype(str).apply(tuple, 1)
    df1_values_in_df2_filter = df1_str_tuples.isin(df2_str_tuples)
    df1_values_not_in_df2 = delta[0][~df1_values_in_df2_filter]
    if len(df1_values_not_in_df2) > 0:
        display(df1_values_not_in_df2.sort_values('service'))
        df2_values_in_df1_filter = df2_str_tuples.isin(df1_str_tuples)
        df2_values_not_in_df1 = delta[i][~df2_values_in_df1_filter]
        display(df2_values_not_in_df1.sort_values('service'))
    else:
        print('No difference.')
    print("\n")

## Add delta columns

So we can see +/- on the min & max easily

In [ ]:
df['delta_min'] = df['minNew'] - df['minOld']
df['delta_max'] = df['maxNew'] - df['maxOld']

## Min Instances Lowered

Find all services where ArcSOC Optimizer recommends lowering the min.

In [ ]:
min_down = df.loc[(df['minOld']>df['minNew'])]
min_down.sort_values('service')

## Min Instances Raised

Find all services where ArcSOC Optimizer recommends raising the min.

In [ ]:
min_up = df.loc[(df['minOld']<df['minNew'])]
min_up.sort_values('service')

## Max Instances Lowered

Find all services where ArcSOC Optimizer recommends lowering the max.

In [ ]:
max_down = df.loc[(df['maxOld']>df['maxNew'])]
max_down.sort_values('service')

## Max Instances Raised

Find all services where ArcSOC Optimizer recommends raising the max.

In [ ]:
max_up = df.loc[(df['maxOld']<df['maxNew'])]
max_up.sort_values('service')


## Instance Math

In [ ]:
# Generate a lookup of combinations of changes to the min & max
# Read the output: where delta_min is X, delta_max has these values, and here is a count of the combination.
grouped = df.groupby(by=['delta_min', 'delta_max']).size()
grouped

In [ ]:
# What is the sum of the recommended changes to min?
print(df['delta_min'].sum())

In [ ]:
# What is the sum of the recommended changes to max?
print(df['delta_max'].sum())